# Build `actions` component

This notebook contains sample code that we will use to generate our `actions` component in the
PSC Live Response orchestration demonstration.

Here we can run `cbapi` code interactively to test out new commands, try new features, and refine
the code before pasting it into an automated script.

## Boilerplate

First we'll start with some logging boilerplate so we can see all DEBUG log output that the
`cbapi` module creates, in case we end up with any errors.

In [ ]:
import logging
logging.basicConfig()
logging.getLogger("cbapi").setLevel(logging.DEBUG)

## Create `CbDefenseAPI` object

For now our platform APIs are hosted in the Defense API namespace. So here we will create an
API key with "Live Response" capability inside the CB PSC console, and save the credentials
to our `cbapi` credential file.

My credential profile is named `eap01-cbtestps` so we will use that as the parameter to the
`CbDefenseAPI` constructor here.

In [ ]:
from cbapi.psc.defense import *
c = CbDefenseAPI(profile="eap01-cbtestps")

## Select the device we want to run on

Now we'll select a device by ID directly. In the final version of our script, we'll get
the device IDs from the `triggers` component by polling for notifications. But for now
we'll just test by going direct to a device we know is online and will accept our
Live Response request.

In [ ]:
dev = c.select(Device, 21212)

## Get some info on this device

As an example of how `cbapi` is 'lazy' - our first time we print out the device info we'll see
there is no information about this device, and in fact, no API call was made to the backend.
We'll call to get the hostname of this device just to demonstrate that API calls will be made
on demand to get the information we need.

Note that we don't have to make any API calls to execute our next command, which will be to 
open the Live Response session to this device. That is, you can safely delete the `dev.name`
line below and the script will continue to function and not make an unnecessary API call to get
device details.

In [ ]:
print(dev)

In [ ]:
dev.name

In [ ]:
print(dev)

## Start the Live Response session

Now we can open a Live Response session to this device. Note that this takes a while- which
is why I clicked 'run all' at the start, before I started droning on describing verbally
each of these code lines.

Hopefully by the time I got here, the session is already open and we can continue on...

In [ ]:
session = dev.lr_session()

## Endpoint Actions

Now that we have an active Live Response session, we can try a few things that we want
our script to accomplish. Let's start by retrieving a file...

In [ ]:
session.get_file("c:\\test.txt")

OK so that worked... let's try starting a few processes and getting the information back
as a string!

In [ ]:
print(session.create_process("ipconfig /all").decode('utf8'))

In [ ]:
print(session.create_process("net start").decode('utf8'))

Finally, let's do some magic and get the list of currently logged-in users... that will feed
our next request to get the Chrome browser history for these logged-in users.

In [ ]:
running_processes = session.list_processes()

# get the list of currently logged-in users
users = set([proc['username'].split('\\')[-1]
             for proc in running_processes if proc['path'].find('explorer.exe') != -1])

print("There are {0} users logged in: {1}".format(len(users), ", ".join(list(users))))

In [ ]:
username = list(users)[0]          # get the first username

history_file = \
                session.get_file("c:\\users\\%s\\appdata\\local\\google\\chrome\\user data\\default\\history"
                                 % username)


## Close the session

And now we're done - let's close the session and take a look at what we acquired!

In [ ]:
session.close()

In [ ]:
len(history_file)

In [ ]:
history_file[:100]